<h2> Install BeautifulSoup </h2>

In [1]:
#install BeautifulSoup and requests
!pip install BeautifulSoup4
!pip install requests

<h2> Scrape Wikipedia page and create Dataframe </h2>

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#get page from wikipedia
link = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(link.text, 'lxml')

#iterate the table to get the data from the page
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert to DataFrame
df = pd.DataFrame(data = data,columns = columns)
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h2> Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
</h2>

In [3]:
#Remove Boroughs that are 'Not assigned'
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h2> More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
</h2>

In [4]:
# Combine multiple neighborhoods into one row separated by comma
df["Neighborhood"] = df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
df = df.drop_duplicates()

#update index 
if(df.index.name != 'Postal Code'):
    df = df.set_index('Postal Code')
    
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h2> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
</h2>

In [5]:
# Not assigned neighborhood, then the neighborhood will be the same as the borough
df['Neighborhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h2> In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
</h2>

In [6]:
df.shape

(103, 2)

<h2> Install wget package </h2>

In [7]:
!pip install wget

<h2> Get the Geospatial Data File </h2>

In [8]:
import wget
url = 'http://cocl.us/Geospatial_data'
filename = wget.download(url)
print(filename)

100% [................................................................................] 2891 / 2891Geospatial_Coordinates (1).csv


<h2> Extract matching latitude and longitude for Postal Code </h2>

In [9]:
#Get data lat/long data from csv
lldf = pd.read_csv(filename)

#rename columns and set the index to be Postcode
lldf.columns = ["Postal Code", "Latitude", "Longitude"]
if(lldf.index.name != 'Postal Code'):
    lldf = lldf.set_index('Postal Code')
    
lldf.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


<h2> Merge DataFrames into one  </h2>

In [10]:
df = df.join(lldf)
df.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
